In [9]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [20]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [21]:
print(y.shape,tX.shape)

(250000,) (250000, 30)


## Calculate the weights with different methods

In [14]:
from implementations import *
from cross_validation import *

In [18]:
# Initialization
max_iters = 50
gamma = 0.2
lambda_=0.5
initial_w=np.zeros(30)

#Pour output un dossier .csv, assurez vous d'avoir train.csv et test.csv dans le même dossier que ce fichier
#Remplacer weightsi par weights si vous voulez utiliser la méthode i (i \in {1,2,3,4})
#Lancer les deux dernières cellues, le dossier en output aura le titre submission.csv
#weights,_=least_squares_GD(y,tX,initial_w,max_iters,gamma)
#weights,_=least_squares_SGD(y,tX,initial_w,max_iters,gamma)
#weights,_=least_squares(y,tX)
weights,_=ridge_regression(y,tX,lambda_)
cross_validation_demo(y,tX,initial_w,max_iters)


ValueError: operands could not be broadcast together with shapes (62500,8,30) (8,) 

In [6]:
print(weights)

[ 2.48519324e-04 -8.96465390e-03 -2.24753247e-03 -2.19688733e-03
 -8.28785856e-04  5.27482033e-04 -1.23580011e-02  2.78401608e-02
  6.37660258e-05  3.17281194e-03 -2.55552723e-02  4.56064526e-02
  1.07748012e-02  6.17277403e-03 -3.38880835e-04 -1.20732147e-03
  2.51582382e-03 -4.25916162e-04  8.32986709e-04  4.90332267e-03
  4.17025376e-04 -7.70891369e-04 -2.48302577e-02  1.45686817e-03
 -7.63838506e-04 -5.43201300e-04  2.59945303e-04  1.61198111e-03
  1.83180261e-04 -5.51499664e-03]


## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
OUTPUT_PATH = 'submission.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)